# **Inteligencja obliczeniowa w analizie danych cyfrowych**

## **Projekt 1: Clumsy Connect Four**

Autorzy: Weronika Hilaszek, Karolina Surówka

### **Opis gry**

 Gra w Connect Four: w **wersji deterministycznej** jest to gra dla dwóch osób wykorzystująca planszę o rozmiarach 7 x 6, polegająca na naprzemiennym wrzucaniu żetonów przez graczy aż do momentu ułożenia czterech żetonów w pionie, poziomie lub po skosie - wygrywa gracz, który pierwszy uzyska taką kombinację. **Wersja probabiltyczna** zakłada, że istnieje prawdopodobieństwo 50%, że żeton wpadnie do wskazanej przez gracza kolumny i 50%, że wpadnie do sąsiedniej kolumny - z prawdopodobieństwem 25%, że wpadnie do kolumny położonej na lewo i 25%, że wpadnie do kolumny położonej na prawo.

In [ ]:
%pip install easyAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 737.9 kB/s eta 0:00:00


In [ ]:
import easyAI
import random
import time
import numpy as np
from easyAI import TwoPlayerGame, AI_Player, Negamax, Human_Player

### Odnalezienie "czwórki"

In [ ]:
def find_four(board, current_player):
    """
    Returns True if the player has connected  4 (or more)
    This is much faster if written in C or Cython
    """
    for pos, direction in POS_DIR:
        streak = 0
        while (0 <= pos[0] <= 5) and (0 <= pos[1] <= 6):
            if board[pos[0], pos[1]] == current_player:
                streak += 1
                if streak == 4:
                    return True
            else:
                streak = 0
            pos = pos + direction
    return False


POS_DIR = np.array(
    [[[i, 0], [0, 1]] for i in range(6)]
    + [[[0, i], [1, 0]] for i in range(7)]
    + [[[i, 0], [1, 1]] for i in range(1, 3)]
    + [[[0, i], [1, 1]] for i in range(4)]
    + [[[i, 6], [1, -1]] for i in range(1, 3)]
    + [[[0, i], [1, -1]] for i in range(3, 7)]
)

### **Implementacja wersji probalistycznej**

W wersji probabilistycznej, wykorzystujemy tablicę z wartościami 0, 1 oraz 2, z której będzie losowana wartość w każdym ruchu. Każda z nich decyduje o kolumnie, do której zostanie wrzucony żeton z danym prawdopodobieństwem - gdy wypadnie 1, żeton wpadnie do wybranej przez gracza kolumny, gdy wypadnie 0 - wpadnie do kolumny po lewej stronie od wybranej, a dla 2 - wpadnie do kolumny po prawej. W przypadku, gdy wylosowana kolumna jest już zapełniona, ruch gracza kończy się brakiem akcji. Istnieją ograniczenia sprawdzające, czy po wylosowaniu ruchu w lewo lub prawo nie wykraczamy poza graniczę planszy.

In [ ]:
## 25% for left, value = 0
## 25% for right, value = 2
## 50% proper column, value = 1

In [ ]:
class ConnectFour_Probabilistic(TwoPlayerGame):

    def __init__(self, players, player, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player = player
        self.move_list = [0,1,1,2] # LEFT: 0, CENTER: 1, RIGHT: 2
        self.moves_number = 0
        random.shuffle(self.move_list)
        print("Starting Agent: ", self.current_player)

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def move_changes(self):
      return random.choice(self.move_list)

    def make_move(self, column):
        change_in_move = self.move_changes()

        if change_in_move == 0 and column > 0:
          column -= 1
        elif change_in_move == 2 and column < 6:
          column += 1

        if column in self.possible_moves():
          line = np.argmin(self.board[:, column] != 0)
          self.board[line, column] = self.current_player

    def show(self):
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )
        self.moves_number +=1

    def lose(self):
        return find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0

### **Implementacja wersji deterministycznej**



W tej implementacji został wykorzystany kod z biblioteki easyAI.

In [ ]:
class ConnectFour_Deterministic(TwoPlayerGame):
    """
    The game of Connect Four, as described here:
    http://en.wikipedia.org/wiki/Connect_Four
    """

    def __init__(self, players, player, board=None):
        self.players = players
        self.board = (
            board
            if (board is not None)
            else (np.array([[0 for i in range(7)] for j in range(6)]))
        )
        self.current_player =  player
        self.moves_number = 0

    def possible_moves(self):
        return [i for i in range(7) if (self.board[:, i].min() == 0)]

    def make_move(self, column):
        line = np.argmin(self.board[:, column] != 0)
        self.board[line, column] = self.current_player

    def show(self):
        print(
            "\n"
            + "\n".join(
                ["0 1 2 3 4 5 6", 13 * "-"]
                + [
                    " ".join([[".", "O", "X"][self.board[5 - j][i]] for i in range(7)])
                    for j in range(6)
                ]
            )
        )
        self.moves_number +=1

    def lose(self):
        return find_four(self.board, self.opponent_index)

    def is_over(self):
        return (self.board.min() > 0) or self.lose()

    def scoring(self):
        return -100 if self.lose() else 0

### **NegaMax bez odcięcia alpha-beta**

W tej implementacji został wykorzystany kod z biblioteki easyAI z pominięciem fragmentów wykorzystujących odcięcie alpha-beta.

In [ ]:
inf = float('infinity')

def negamax_without_alpha_beta(game, depth, origDepth, scoring):
    if (depth == 0) or game.is_over():
        return scoring(game) * (1 + 0.001 * depth)

    possible_moves = game.possible_moves()

    state = game
    best_move = possible_moves[0]
    if depth == origDepth:
        state.ai_move = possible_moves[0]

    bestValue = -inf
    unmake_move = hasattr(state, 'unmake_move')


    for move in possible_moves:

        if not unmake_move:
            game = state.copy() # re-initialize move

        game.make_move(move)
        game.switch_player()

        move_alpha = - negamax_without_alpha_beta(game, depth-1, origDepth, scoring)

        if unmake_move:
            game.switch_player()
            game.unmake_move(move)

        if bestValue < move_alpha:
            bestValue = move_alpha
            best_move = move

    return bestValue

class Negamax_without_alpha_beta:
    def __init__(self, depth, scoring=None, win_score=+inf):
        self.scoring = scoring
        self.depth = depth
        self.win_score= win_score



    def __call__(self,game):
        scoring = self.scoring if self.scoring else (
                       lambda g: g.scoring() )

        self.alpha = negamax_without_alpha_beta(game, self.depth, self.depth, scoring)
        return game.ai_move

# Uruchamianie

Napisz kod, który uruchamia dwóch graczy AI z algorytmem Negamax przeciwko sobie wielokrotnie, zmieniając gracza rozpoczynającego. Policz liczbę zwycięstw każdego z graczy. Porównaj dwa różne ustawienia maksymalnej głębokości dla gier deterministycznych na deterministycznym i probabilistycznym wariancie Twojej gry.

Prezentujemy przykładowe 2 rozgrywki dla wariantów z taką samą głębokością dla obu graczy, a na koniec zestawiamy różne uzyskane wyniki dla różnych kombinacji.



#### Wersja deterministyczna z NegaMaxem z odcięciem alpha-beta

In [ ]:
if __name__ == "__main__":
    # LET'S PLAY !

    wins_player1 = 0
    wins_player2 = 0
    num_games = 2
    times = []
    number_of_moves =[]

    ai_algo_neg = Negamax(3)
    ai_algo_neg2 = Negamax(3)

    start_player1 = True

    for _ in range(num_games):
      if start_player1:
        player = 1
      else:
        player = 2
      game = ConnectFour_Deterministic([AI_Player(ai_algo_neg), AI_Player(ai_algo_neg2)], player = player)
      start = time.time()
      game.play()
      duration = time.time() - start
      if game.lose():
          print("Player %d wins." % (game.opponent_index))
          if game.opponent_index == 1:
                wins_player1 += 1
          else:
              wins_player2 += 1
      else:
          print("Looks like we have a draw.")

      print("Single time: ", duration)
      times.append(duration)
      print("Number of moves: ", game.moves_number-1)
      number_of_moves.append(game.moves_number-1)
      start_player1 = not start_player1

    print("------------------------------------")
    print("Number of played games: ", num_games)
    print(f"Player 1 wins: {wins_player1} times")
    print(f"Player 2 wins: {wins_player2} times")
    [ print(i) for i in times]
    # print("Time: ", time)
    print("Total time: ", np.sum(times))
    print("Total number of moves: ",np.sum(number_of_moves))
    print("Avarage time for move: ", (np.sum(times)/np.sum(number_of_moves)))
    print("------------------------------------")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Move #1: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .

Move #2: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .

Move #3: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #4: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #5: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #6: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #7: player 1 plays 1 :

0 1 2

### Wersja Probabilistyczna z Negamaxem z odcięciem alpha-beta

In [ ]:
if __name__ == "__main__":
    # LET'S PLAY !

    wins_player1 = 0
    wins_player2 = 0
    num_games = 2
    times = []
    number_of_moves =[]

    ai_algo_neg = Negamax(3)
    ai_algo_neg2 = Negamax(3)

    start_player1 = True

    for _ in range(num_games):
      if start_player1:
        player = 1
      else:
        player = 2
      start = time.time()
      game = ConnectFour_Probabilistic([AI_Player(ai_algo_neg), AI_Player(ai_algo_neg2)], player = player)
      game.play()
      duration = time.time() - start
      if game.lose():
          print("Player %d wins." % (game.opponent_index))
          if game.opponent_index == 1:
                wins_player1 += 1
          else:
              wins_player2 += 1
      else:
          print("Looks like we have a draw.")

      print("Single time: ", duration)
      times.append(duration)
      print("Number of moves: ", game.moves_number-1)
      number_of_moves.append(game.moves_number-1)
      start_player1 = not start_player1

    print("------------------------------------")
    print("Number of played games: ", num_games)
    print(f"Player 1 wins: {wins_player1} times")
    print(f"Player 2 wins: {wins_player2} times")
    [ print(i) for i in times]
    # print("Time: ", time)
    print("Total time: ", np.sum(times))
    print("Total number of moves: ",np.sum(number_of_moves))
    print("Avarage time for move: ", (np.sum(times)/np.sum(number_of_moves)))
    print("------------------------------------")

Starting Agent:  1

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Move #1: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .

Move #2: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .

Move #3: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #4: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O X . . . . .

Move #5: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
O . . . . . .
O . . . . . .
X . . . . . .
O X . . . . .

Move #6: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
X . . . . . .
O . . . . . .
O . . . . . .
X . . . . . .
O X . . . . .

Move #7: player

### **Statystyki dla wariantu deterministycznego i probabilistycznego dla różnych ustawień głębokości:**



**WERSJE Z ALPHA-BETA ODCIĘCIEM**

Liczba gier: 20  
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 5  
Maksymalna głębokość dla 2 gracza: 5  

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 0     | 12    |
| wygrane 2 gracza   | 20   | 8     |  


---



Liczba gier: 20  
Zaczyna gracz: 2  
Maksymalna głębokość dla 1 gracza: 5  
Maksymalna głębokość dla 2 gracza: 5  

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 20     | 8     |
| wygrane 2 gracza  | 0     | 12  |


---



Liczba gier: 20     
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 7  
Maksymalna głębokość dla 2 gracza: 7

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 0    | 10    |
| wygrane 2 gracza  | 20     | 10     |


---



Liczba gier: 20  
Zaczyna gracz: 2  
Maksymalna głębokość dla 1 gracza: 7  
Maksymalna głębokość dla 2 gracza: 7  

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 20     | 10     |
| wygrane 2 gracza  | 0     | 10     |


---




Liczba gier: 10  
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 5  
Maksymalna głębokość dla 2 gracza: 7

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 0    | 4    |
| wygrane 2 gracza  | 10     | 6    |


---




Liczba gier: 10  
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 7  
Maksymalna głębokość dla 2 gracza: 5

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 10     | 5     |
| wygrane 2 gracza  | 0     | 5     |


---




Liczba gier: 10  
Zaczyna gracz: 2  
Maksymalna głębokość dla 1 gracza: 5  
Maksymalna głębokość dla 2 gracza: 7

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 10     | 6     |
| wygrane 2 gracza  | 0     | 4     |


---




Liczba gier: 10  
Zaczyna gracz: 2  
Maksymalna głębokość dla 1 gracza: 7  
Maksymalna głębokość dla 2 gracza: 5

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 10     | 5     |
| wygrane 2 gracza  | 0     | 5    |



---


Porównaj kilka (co najmniej Negamax z i bez odcięcia alfa-beta, z dwoma różnymi ustawieniami maksymalnej głębokości algorytmy dla gier deterministycznych na deterministycznych i probabilistycznych wariantach twojej gry. Napisz kod, który mierzy średni czas spędzony na wybieraniu akcji przez każdy wariant AI Dodaj zmierzone czasy do raportu.

#### Wersja Probabilistyczna z NegaMaxem bez odcięcia alpha-beta

Poniżej prezentujemy przykładowe rozgrywki dla wariantu deterministycznego i probabilistycznego bez odcięcia alpha-beta, a na koniec umieszczamy statystyki dla różnych kombinacji głębokości ze zmianą gracza rozpoczynającego oraz średnie czasy zmierzone dla wszystkich zaprezentowanych wariantów.

In [ ]:
if __name__ == "__main__":
    # LET'S PLAY !

    wins_player1 = 0
    wins_player2 = 0
    num_games = 1
    times = []
    number_of_moves =[]

    ai_algo_neg = Negamax_without_alpha_beta(3)
    ai_algo_neg2 = Negamax_without_alpha_beta(3)

    start_player1 = True

    for _ in range(num_games):
      if start_player1:
        player = 1
      else:
        player = 2
      start = time.time()
      game = ConnectFour_Probabilistic([AI_Player(ai_algo_neg), AI_Player(ai_algo_neg2)], player = player)
      game.play()
      duration = time.time() - start
      if game.lose():
          print("Player %d wins." % (game.opponent_index))
          if game.opponent_index == 1:
                wins_player1 += 1
          else:
              wins_player2 += 1
      else:
          print("Looks like we have a draw.")

      print("Single time: ", duration)
      times.append(duration)
      print("Number of moves: ", game.moves_number-1)
      number_of_moves.append(game.moves_number-1)
      start_player1 = not start_player1

    print("------------------------------------")
    print("Number of played games: ", num_games)
    print(f"Player 1 wins: {wins_player1} times")
    print(f"Player 2 wins: {wins_player2} times")
    [ print(i) for i in times]
    # print("Time: ", time)
    print("Total time: ", np.sum(times))
    print("Total number of moves: ",np.sum(number_of_moves))
    print("Avarage time for move: ", (np.sum(times)/np.sum(number_of_moves)))
    print("------------------------------------")

Starting Agent:  2

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Move #1: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .

Move #2: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .

Move #3: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .

Move #4: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .

Move #5: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .

Move #6: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .

Move #7: player

#### Wersja deterministyczna z NegaMaxem bez odcięcia alpha-beta

In [ ]:
if __name__ == "__main__":
    # LET'S PLAY !

    wins_player1 = 0
    wins_player2 = 0
    num_games = 1
    times = []
    number_of_moves =[]

    ai_algo_neg = Negamax_without_alpha_beta(3)
    ai_algo_neg2 = Negamax_without_alpha_beta(3)

    start_player1 = True

    for _ in range(num_games):
      if start_player1:
        player = 1
      else:
        player = 2
      start = time.time()
      game = ConnectFour_Deterministic([AI_Player(ai_algo_neg), AI_Player(ai_algo_neg2)], player = player)
      game.play()
      duration = time.time() - start
      if game.lose():
          print("Player %d wins." % (game.opponent_index))
          if game.opponent_index == 1:
                wins_player1 += 1
          else:
              wins_player2 += 1
      else:
          print("Looks like we have a draw.")

      print("Single time: ", duration)
      times.append(duration)
      print("Number of moves: ", game.moves_number-1)
      number_of_moves.append(game.moves_number-1)
      start_player1 = not start_player1

    print("------------------------------------")
    print("Number of played games: ", num_games)
    print(f"Player 1 wins: {wins_player1} times")
    print(f"Player 2 wins: {wins_player2} times")
    [ print(i) for i in times]
    # print("Time: ", time)
    print("Total time: ", np.sum(times))
    print("Total number of moves: ",np.sum(number_of_moves))
    print("Avarage time for move: ", (np.sum(times)/np.sum(number_of_moves)))
    print("------------------------------------")


0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .

Move #1: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .

Move #2: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .

Move #3: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #4: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
. . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #5: player 1 plays 0 :

0 1 2 3 4 5 6
-------------
. . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #6: player 2 plays 0 :

0 1 2 3 4 5 6
-------------
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .
X . . . . . .
O . . . . . .

Move #7: player 1 plays 1 :

0 1 2

**WERSJE BEZ ALPHA-BETA ODCIĘCIA**

Liczba gier: 10  
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 5  
Maksymalna głębokość dla 2 gracza: 5  

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 10     | 4     |
| wygrane 2 gracza  | 0     | 5    |

Jeden remis dla wersji probilistycznej - 54 ruchy


---



Liczba gier: 10  
Zaczyna gracz: 2   
Maksymalna głębokość dla 1 gracza: 5  
Maksymalna głębokość dla 2 gracza: 5  

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 0    | 5     |
| wygrane 2 gracza  | 10    | 5    |


---



Liczba gier: 10   
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 3   
Maksymalna głębokość dla 2 gracza: 3

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 10     | 7     |
| wygrane 2 gracza  | 0     | 3     |


---



Liczba gier: 10  
Zaczyna gracz: 2  
Maksymalna głębokość dla 1 gracza: 3  
Maksymalna głębokość dla 2 gracza: 3  

| | wariant deterministyczny | wariant probabilistyczny |
|------------|------------|------------|
| wygrane 1 gracza  | 0     | 6     |
| wygrane 2 gracza  | 10     | 4     |


---




Liczba gier: 10  
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 5  
Maksymalna głębokość dla 2 gracza: 3

| | wariant deterministyczny |
|------------|------------|
| wygrane 1 gracza  | 10     |
| wygrane 2 gracza  | 0     |


---




Liczba gier: 10  
Zaczyna gracz: 1  
Maksymalna głębokość dla 1 gracza: 3  
Maksymalna głębokość dla 2 gracza: 5

| | wariant deterministyczny |
|------------|------------|
| wygrane 1 gracza  | 10     |
| wygrane 2 gracza  | 0    |

### **Średnie uzyskane czasy dla różnych rozgrywek i wariantów**

Wariant deterministyczny z odcięciem alpha-beta:
- dla głębokości 5 dla obu graczy: 0.10728737630342182
- dla głębokości 7 dla obu graczy: 0.6379378499169098
- dla głębokości 5 i 7: 0.33834266976306315

Wariant probabilistyczny z odcięciem alpha-beta:
- dla głębokości 5 dla obu graczy: 0.1911993986027008
- dla głębokości 7 dla obu graczy: 1.3482893561323483
- dla głębokości 5 i 7: 0.7597329410639676

Wariant deterministyczny bez odcięcia alpha-beta:
- dla głębokości 3 dla obu graczy: 0.09963440267663252
- dla głębokości 5 dla obu graczy: 4.015394178189729
- dla głębokości 3 i 5: 2.331358754984006

Wariant probalistyczny bez odcięcia alpha-beta:
- dla głębokości 3 dla obu graczy: 0.13411827648387237
- dla głębokości 5 dla obu graczy: 3.8646183121473268

### **Wnioski**

Znaczącą różnicę można zauważyć w długości trwania rozgrywek z oraz bez odcięcia alpha-beta. Niezaprzeczalnie zastosowanie odcięcia alpha-beta przyśpiesza i usprawnia przeprowadzenie rozgrywek.

Im większa głębokość dla danego gracza, tym dłuższy czas wykonywania.

Dla wariantu deterministycznego, z odcięciem alpha-beta zawsze przegrywa ten, który zaczął - bez odcięcia występuje sytuacja odwotna.

Dla wariantu probabilistycznego, szanse na wygraną są bardziej wyrównane.